In [6]:
!pip install langchain
!pip install pypdf
!pip install Transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.2 MB/s eta 0:00:00


In [72]:
import os
os.environ['VECTARA_API_KEY'] = 'VECTARA_API_KEY'
os.environ['VECTARA_CORPUS_ID'] = 'VECTARA_CORPUS_ID'
os.environ['VECTARA_CUSTOMER_ID']='VECTARA_CUSTOMER_ID'


In [73]:
vectara_corpus_id = os.environ['VECTARA_CORPUS_ID']
vectara_customer_id = os.environ['VECTARA_CUSTOMER_ID']
vectara_api_key = os.environ['VECTARA_API_KEY']

In [9]:
from langchain import HuggingFaceHub

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import vectara
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Vectara

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

from getpass import getpass

In [13]:
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [14]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [77]:
loader = DirectoryLoader("/content/folder", glob = "./*.pdf", loader_cls = PyPDFLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [78]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id = repo_id, model_kwargs= {"temperature": 0.5, "max_length": 10000,"max_new_tokens": 3000})

In [79]:
vectorstore = Vectara(
                vectara_customer_id=vectara_customer_id,
                vectara_corpus_id=vectara_corpus_id,
                vectara_api_key=vectara_api_key
            )

In [80]:
vectara = Vectara.from_documents(
    docs,
    embedding=None,
)


No permissions to add document to Vectara. 
                Check your corpus ID, customer ID and API key


In [81]:
starting_tag = "<s>[INST]"
ending_tag = "</s>[INST]"
prompt_template = """You are an AI assisstant which helps the people to balance their diet by prvoding them diet plans.
                      You need to provide the diet plans. which should include breakfast, lunch and dinner
                  Context :{context}
                  Question: {question}"""

prompt = starting_tag + prompt_template + ending_tag
PROMPT = PromptTemplate(template=prompt, input_variables=["context","question"])
chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = vectara.as_retriever(),
        input_key = "question",
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents = False
    )

In [83]:

response = chain(input("Any diet questions or thoughts you'd like to share?  "))
if "result" in response:
  result = response["result"]
  parts = result.split("</s>[INST]")
  if len(parts) > 1:

    answer = parts[1].strip()
    print(answer)
  else:
    print("Not Found")
# response

Any diet questions or thoughts you'd like to share?  Give me 3 days diet plans. I am 20 year old men
I'm a 20-year-old man. Here are three different diet plans based on the healthy US-style dietary pattern for your age group. Each plan provides approximately 2000 calories per day. Feel free to adjust portion sizes based on your personal hunger and activity level.

Plan 1:
- Breakfast: 2 scrambled eggs, 1 slice whole grain toast, 1 medium banana, 1 small apple
- Lunch: 3 oz grilled chicken breast, 1 cup cooked brown rice, 1 cup steamed broccoli, 1 medium carrot
- Snack: 1 medium pear, 1 oz mixed nuts
- Dinner: 3 oz broiled salmon, 1 cup cooked quinoa, 1 cup roasted sweet potatoes, 1 cup green beans
- Dessert: 1 small apple sliced with 1 tbsp peanut butter

Plan 2:
- Breakfast: 1 cup cooked oatmeal topped with 1 cup sliced strawberries and 1 tbsp honey, 1 slice whole grain toast
- Lunch: 3 oz grilled turkey, 1 cup cooked black beans, 1 cup cooked brown rice, 1 medium bell pepper
- Snack: